# Importing Data

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.max_columns = 6
pd.__version__

u'0.17.0'

We often times have a variety of input data.

- CSV
- Excel
- SQL
- HDF5

This is subset of the data from beeradvocate.com, via [Standford](https://snap.stanford.edu/data/web-RateBeer.html). It's strangely formatted.

```
beer/name: Sausa Weizen
beer/beerId: 47986
beer/brewerId: 10325
beer/ABV: 5.00
beer/style: Hefeweizen
review/appearance: 2.5
review/aroma: 2
review/palate: 1.5
review/taste: 1.5
review/overall: 1.5
review/time: 1234817823
review/profileName: stcules
review/text: A lot of foam. But a lot.	In the smell some banana, and then lactic and tart. Not a good start.	Quite dark orange in color, with a lively carbonation (now visible, under the foam).	Again tending to lactic sourness.	Same for the taste. With some yeast and banana.		

beer/name: Red Moon
beer/beerId: 48213
beer/brewerId: 10325
beer/ABV: 6.20
 ...
```

The dataset was a bit large to processess all at once

```bash
$ wc -l beeradvocate.txt
 22212596 beeradvocate.txt
```

```python
def format_review(review):
    return dict(map(lambda x: x.strip().split(": ", 1), review))

def as_dataframe(reviews):
    col_names = {
        'beer/ABV': 'abv', 'beer/beerId': 'beer_id', 'beer/brewerId': 'brewer_id',
        'beer/name': 'beer_name', 'beer/style': 'beer_style',
        'review/appearance': 'review_appearance', 'review/aroma': 'review_aroma',
        'review/overall': 'review_overll', 'review/palate': 'review_palate',
        'review/profileName': 'profile_name', 'review/taste': 'review_taste',
        'review/text': 'text', 'review/time': 'time'
    }
    df = pd.DataFrame(list(reviews))
    numeric = ['abv', 'review_appearance', 'review_aroma',
               'review_overall', 'review_palate', 'review_taste']
    df = (df.rename(columns=col_names)
            .replace('', np.nan))
    df[numeric] = df[numeric].astype(float)
    df['time'] = pd.to_datetime(df.time.astype(int), unit='s')
    return df

def main():
    with open('beeradvocate.txt') as f:
        reviews = filter(lambda x: x != ('\n',),
                         partitionby(lambda x: x == '\n', f))
        reviews = map(format_review, reviews)
        reviews = partition(100000, reviews, pad=None)
        reviews = filter(None, reviews)
        os.makedirs('beer_reviews', exist_ok=True)

        for i, subset in enumerate(reviews):
            print(i, end='\r')
            df = as_dataframe(subset)
            df.to_csv('beer_reviews/review_%s.csv' % i, index=False)
```

# Bootstrap

Let's get the data

In [4]:
df = pd.read_csv('data/beer2.csv.gz', index_col=0, parse_dates=['time'])

# CSV

In [5]:
df = pd.read_csv('data/beer.csv', index_col=0, parse_dates=['time'])

In [6]:
df.to_csv('tmp/beer.csv')

# Excel

In [7]:
df = pd.read_excel('data/beer.xls', sheetnames=[0])

In [8]:
df.to_excel('tmp/beer.xls')

# SQL

odo library

http://odo.readthedocs.org/en/latest/


In [9]:
from odo import odo
df = odo('sqlite:///data/beer.sqlite::table', pd.DataFrame)

In [10]:
!rm tmp/beer.sqlite
odo(df, 'sqlite:///tmp/beer.sqlite::table')

Table('table', MetaData(bind=Engine(sqlite:///tmp/beer.sqlite)), Column('abv', Float(precision=53), table=<table>), Column('beer_id', BigInteger(), table=<table>, nullable=False), Column('brewer_id', BigInteger(), table=<table>, nullable=False), Column('beer_name', Text(), table=<table>), Column('beer_style', Text(), table=<table>), Column('review_appearance', Float(precision=53), table=<table>), Column('review_aroma', Float(precision=53), table=<table>), Column('review_overall', Float(precision=53), table=<table>), Column('review_palate', Float(precision=53), table=<table>), Column('profile_name', Text(), table=<table>), Column('review_taste', Float(precision=53), table=<table>), Column('text', Text(), table=<table>), Column('time', DateTime(), table=<table>), schema=None)

# HDF

In [11]:
df = pd.read_hdf('data/beer.hdf','df')

In [13]:
df.to_hdf('data/beer.hdf','df',mode='w',format='fixed')

# Timings

In [14]:
%timeit pd.read_excel('data/beer.xls', sheetnames=[0])

1 loops, best of 3: 2.76 s per loop


In [16]:
%timeit odo('sqlite:///data/beer.sqlite::table', pd.DataFrame)

1 loops, best of 3: 958 ms per loop


In [17]:
%timeit pd.read_csv('data/beer2.csv', index_col=0, parse_dates=['time'])

1 loops, best of 3: 559 ms per loop


In [18]:
%timeit pd.read_hdf('data/beer.hdf','df')

1 loops, best of 3: 273 ms per loop
